Sources:
https://platform.openai.com/docs/guides/fine-tuning

In [1]:
import yaml
import random

In [2]:
with open('Cat_0_Data/data/Cat_0_commonexample.yaml') as f:
    data = yaml.safe_load(f)

In [3]:
def parse_examples(examples: str) -> list[str]:
    """Converts examples from YAML file to list of utterance texts

    Args:
        examples (string): Examples from YAML data file.
                           It is a single string containing all utterances
                           separated by '\n- '

    Returns:
        List[str]: list of utterance texts
    """

    texts = [text.strip() for text in examples.lstrip("- ").split("\n- ")]
    # splitting by '\n -' causes empty string to be present in the list,
    # in case the value of examples is ''.
    if "" in texts:
        texts.remove("")
    return texts

A prompt separator string needs to be added at the end of the prompt both while fine tuning and sending requests to the model. Else, the outputted completions would most likely be random, instead of our desired output intents. This separator should NOT be present in the utterance at all.

In [8]:
prompt_separator = "\n\nIntent:\n\n"

Completions should start with a whitespace since OpenAI's tokenization tokenizes most words with a preceding whitespace.

For classification tasks, OpenAI recommends us to choose classes which map to a single token and set max_tokens=1. This is also a must when we want the model to output classification metrics. But some classes like "Enquiry" or "Incident" map to multiple tokens. <br>
Usually, we set a stop sequence at the end of the completion instead. But this won't work if we want to calculate classification metrics. So we convert the intent to strings of one token.

In [ ]:
intent_completion_mapping = {
    'Incident': ' Incident',
    'Request': ' Request',
    'Enquiry': ' En'
}

In [9]:
train_list = []
val_list = []

for intent_dict in data['nlu']:
    intent = intent_dict['intent']
    examples = parse_examples(intent_dict['examples'])
    random.shuffle(examples)

    train_set_size = int(0.8 * len(examples))
    
    for example in examples[:train_set_size]:
        train_list.append({'prompt': f'{example}{prompt_separator}', 
                          'completion': intent_completion_mapping[intent]})
    for example in examples[train_set_size:]:
        val_list.append({'prompt': f'{example}{prompt_separator}', 
                          'completion': intent_completion_mapping[intent]})

In [11]:
len(train_list), len(val_list)

(4349, 1089)

In [12]:
import jsonlines

In [13]:
with jsonlines.open('data_train.jsonl', mode='w') as writer:
    # Write each data item as a separate line
    for item in train_list:
        writer.write(item)

In [14]:
with jsonlines.open('data_val.jsonl', mode='w') as writer:
    # Write each data item as a separate line
    for item in val_list:
        writer.write(item)

In [4]:
import dotenv
dotenv.load_dotenv(dotenv.find_dotenv())

True

In [16]:
!openai api fine_tunes.create \
  -t  data_train.jsonl\
  -v data_val.jsonl \
  -m ada \
  --compute_classification_metrics \
  --classification_n_classes 3


Upload progress: 100%|███████████████████████| 560k/560k [00:00<00:00, 541Mit/s]
Uploaded file from data_train.jsonl: file-Rv7Dzp9MFx7xUlKZ0UJKKuy6
Upload progress: 100%|███████████████████████| 140k/140k [00:00<00:00, 115Mit/s]
Uploaded file from data_val.jsonl: file-LAL3AwKQ6mFR8Ms1RMMbNfrH
Created fine-tune: ft-nzoSpioROVwjAgEktgSHotLD
Streaming events until fine-tuning is complete...

(Ctrl-C will interrupt the stream, but not cancel the fine-tune)
[2023-07-03 21:55:26] Created fine-tune: ft-nzoSpioROVwjAgEktgSHotLD

Stream interrupted (client disconnected).
To resume the stream, run:

  openai api fine_tunes.follow -i ft-nzoSpioROVwjAgEktgSHotLD



In [13]:
!openai api fine_tunes.follow -i ft-nzoSpioROVwjAgEktgSHotLD

[2023-07-03 21:55:26] Created fine-tune: ft-nzoSpioROVwjAgEktgSHotLD
[2023-07-03 22:54:52] Fine-tune costs $0.17
[2023-07-03 22:54:52] Fine-tune enqueued. Queue number: 17
[2023-07-03 22:55:09] Fine-tune is in the queue. Queue number: 16
[2023-07-03 22:55:36] Fine-tune is in the queue. Queue number: 15
[2023-07-03 22:55:38] Fine-tune is in the queue. Queue number: 14
[2023-07-03 22:55:53] Fine-tune is in the queue. Queue number: 13
[2023-07-03 22:56:28] Fine-tune is in the queue. Queue number: 12
[2023-07-03 22:57:33] Fine-tune is in the queue. Queue number: 11
[2023-07-03 22:57:51] Fine-tune is in the queue. Queue number: 10
[2023-07-03 22:57:55] Fine-tune is in the queue. Queue number: 9
[2023-07-03 22:58:43] Fine-tune is in the queue. Queue number: 8
[2023-07-03 22:59:46] Fine-tune is in the queue. Queue number: 7
[2023-07-03 22:59:47] Fine-tune is in the queue. Queue number: 6
[2023-07-03 23:00:30] Fine-tune is in the queue. Queue number: 5
[2023-07-03 23:01:04] Fine-tune is in the

In [14]:
!openai api fine_tunes.results -i ft-nzoSpioROVwjAgEktgSHotLD

step,elapsed_tokens,elapsed_examples,training_loss,training_sequence_accuracy,training_token_accuracy,validation_loss,validation_sequence_accuracy,validation_token_accuracy,classification/accuracy,classification/weighted_f1_score
1,328,8,0.5633108771260349,0.0,0.0,0.5201181193918684,0.0,0.0,,
2,912,16,0.2980736950701774,0.0,0.0,,,,,
3,1368,24,0.29758118203852446,0.0,0.0,,,,,
4,1760,32,0.2778889345377244,0.0,0.0,,,,,
5,2536,40,0.09113537419199928,0.125,0.125,,,,,
6,2992,48,0.08846677964001044,0.5,0.5,,,,,
7,3384,56,0.07418904255941015,0.5,0.5,,,,,
8,3904,64,0.05748189510424708,0.125,0.125,,,,,
9,4424,72,0.0547062827908303,0.25,0.25,0.031127390889326177,0.75,0.75,,
10,4880,80,0.049412650189395724,0.375,0.375,,,,,
11,5208,88,0.0653977513703867,0.25,0.25,,,,,
12,5600,96,0.049180028506161766,0.375,0.375,,,,,
13,5928,104,0.059193774577230206,0.5,0.5,,,,,
14,6320,112,0.053975621091400784,0.375,0.375,,,,,
15,6648,120,0.05327690430462826,0.5,0.5,,,,,
16,6912,128,0.06079243592947023,0.625,0.625,

1169,497032,9352,0.01734790824651479,1.0,1.0,0.018619143849900485,1.0,1.0,,
1170,497424,9360,0.02246301004171092,1.0,1.0,,,,,
1171,497944,9368,0.011610138174168868,1.0,1.0,,,,,
1172,498272,9376,0.016191470408947226,1.0,1.0,,,,,
1173,498664,9384,0.016173448341700505,1.0,1.0,,,,,
1174,498928,9392,0.01144722069737071,1.0,1.0,,,,,
1175,499256,9400,0.024508356274614472,0.875,0.875,,,,,
1176,499520,9408,0.019576188270642662,1.0,1.0,,,,,
1177,499912,9416,0.017996868820406842,1.0,1.0,0.012293916151858689,1.0,1.0,,
1178,500240,9424,0.01579696304581819,1.0,1.0,,,,,
1179,500504,9432,0.02896363952469073,0.875,0.875,,,,,
1180,500960,9440,0.015711804796460945,1.0,1.0,,,,,
1181,501352,9448,0.011863502199948925,0.875,0.875,,,,,
1182,501744,9456,0.01051738091178938,1.0,1.0,,,,,
1183,502264,9464,0.01244444300541119,1.0,1.0,,,,,
1184,502912,9472,0.015789206042064394,0.875,0.875,,,,,
1185,503240,9480,0.02025301656070633,1.0,1.0,0.05943911712915633,0.625,0.625,,
1186,503504,9488,0.01897636769732031,1.0,1.0

In [12]:
import openai
response = openai.Completion.create(
    model="ada:ft-personal-2023-07-03-18-02-46",
    prompt=f"Got logged out of my account{prompt_separator}",
    max_tokens=1
)
print(response['choices'][0]['text'].strip())

Incident
